# Subword Tokenization

In [1]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [2]:
token_ids = enc.encode('Prüfungsvorleistung')
token_ids

[3617, 2448, 79706, 3576, 269, 273, 84314]

In [3]:
[enc.decode([i]) for i in token_ids]

['Pr', 'ü', 'fung', 'sv', 'or', 'le', 'istung']

# Phenotyping with LLMs

We will show how to use ChatGPT through the OpenAPI API for zero-shot and few-shot smoking status classification, which is a kind of phenotyping task. Note: if you want to run the notebook yourself, make sure to provide an API key: https://github.com/openai/openai-python#usage

In [4]:
from openai import OpenAI
import os

client = OpenAI()

In [5]:
# Helper function to send messages to OpenAI API (ChatGPT model)
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content.replace('```', '')

## Zero-Shot Inference

In [6]:
text = "Social History: No alcohol use and quit tobacco greater than 25 years ago with a 10-pack year smoking history."

### Prompt 1

Describes the task

In [7]:
prompt1 = "What is the smoking status of the person described in this clinical note? ```{}```"

In [8]:
prompt1.format(text)

'What is the smoking status of the person described in this clinical note? ```Social History: No alcohol use and quit tobacco greater than 25 years ago with a 10-pack year smoking history.```'

In [9]:
get_completion(prompt1.format(text))

'The smoking status of the person described in this clinical note is that they quit tobacco greater than 25 years ago.'

### Prompt 2

Describes the task and valid response options (for classification)

In [10]:
prompt2 = ("What is the smoking status of the person described in this clinical note?"
" The valid options are: smoker, non-smoker, ex-smoker "
" Input: ```{}```")

In [11]:
get_completion(prompt2.format(text))

'The smoking status of the person described in this clinical note is "ex-smoker".'

### Prompt 3

Describes the task, valid response options, and output format (JSON)

In [12]:
prompt3 = ("What is the smoking status of the person described in this clinical note?"
" The valid options are: current smoker, non-smoker, ex-smoker "
" Please return the answer as a JSON of the format {{ label : <label> }} without any explanations."
" Input: ```{}```")

In [13]:
response = get_completion(prompt3.format(text))
response

'{"label": "ex-smoker"}'

In [14]:
import json
json.loads(response)

{'label': 'ex-smoker'}

### Prompt 4

In addition to the previous prompt, we ask the model to perform an annotational task, i.e., retrieving a piece of text that justifies the classification

In [15]:
prompt4 = ("What is the smoking status of the person described in this clinical note?"
" The valid options are: current smoker, non-smoker, ex-smoker "
" Please return the answer as a JSON of the format {{ label : <label>, evidence: <keyphrase> }} without any explanations. "
" 'evidence' should contain the shortest possible substring from the input that can be used to justify the label."
" Input: ```{}```")

In [16]:
get_completion(prompt4.format(text))

'{"label": "ex-smoker", "evidence": "quit tobacco greater than 25 years ago"}'

## In-Context Learning

Instead of describing the task in detail, we can provide some training example inside the prompt

In [17]:
prompt_few_shot = ('Your task is to determine the smoking status of the person described in a clinical note.\n'
'Please return the answer as a JSON of the format {{ label : <label>, evidence: <keyphrase> }} without any explanations.\n'
'Here are some examples:\n'
'Input: ```Smoker until 1999``` Output: ```{{ "label" : "ex-smoker", "keyphrase": "Smoker until 1999"}}```\n'
'Input: ```… SOCIAL HISTORY: Widowed since 1972, no tobacco, no alcohol, lives alone.``` Output: ```{{ "label" : "non-smoker", "keyphrase": "no tobacco"}}```\n'
'Input: ```He is a heavy smoker and drinks 2–3 shots per day at times.``` Output: ```{{ "label" : "current smoker", "keyphrase": "heavy smoker"}}```\n'
'Input: ```{}``` Output: ')

In [18]:
print(prompt_few_shot.format(text))

Your task is to determine the smoking status of the person described in a clinical note.
Please return the answer as a JSON of the format { label : <label>, evidence: <keyphrase> } without any explanations.
Here are some examples:
Input: ```Smoker until 1999``` Output: ```{ "label" : "ex-smoker", "keyphrase": "Smoker until 1999"}```
Input: ```… SOCIAL HISTORY: Widowed since 1972, no tobacco, no alcohol, lives alone.``` Output: ```{ "label" : "non-smoker", "keyphrase": "no tobacco"}```
Input: ```He is a heavy smoker and drinks 2–3 shots per day at times.``` Output: ```{ "label" : "current smoker", "keyphrase": "heavy smoker"}```
Input: ```Social History: No alcohol use and quit tobacco greater than 25 years ago with a 10-pack year smoking history.``` Output: 


In [19]:
response = get_completion(prompt_few_shot.format(text))
response

'{ "label" : "ex-smoker", "keyphrase": "quit tobacco greater than 25 years ago" }'

### A More Complicated Example

We will use a German-language case report from: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7988258/

In [20]:
german_text_long = """
Fallschilderung
Anamnese. Der 67-jährige Patient ist bekannt in Ihrer allgemeininternistischen Praxis.
Letzte Vorstellung vor 2 Monaten: Erstdiagnose Magenkarzinom, Beginn neoadjuvante Chemotherapie (FLOT-Protokoll mit 5‑Fluorouracil, Folinsäure, Oxaliplatin und Docetaxel) in potenziell kurativem Setting im lokalen Klinikum
Heutige Vorstellung: seit etwa 10 Tagen zunehmende Dyspnoe (erst unter Belastung, mittlerweile auch in Ruhe, Orthopnoe nachts).
Gelegentlich etwas trockener Husten, zweimalig Temperatur von 37,7 °C in den letzten 10 Tagen.
In 3 Tagen steht der nächste Chemotherapiezyklus an. Der Patient legt einen Arztbrief vor.
Inhalt: stationäre Behandlung vor 3 Wochen aufgrund einer Pneumonie (linksseitig)

Vorerkrankungen. Linksherzinsuffizienz („heart failure with preserved ejection fraction“ [HFpEF], linksventrikuläre Ejektionsfraktion 50 %), chronisch-obstruktive Lungenerkrankung im Stadium I nach Global Initiative for Chronic Obstructive Lung Disease (GOLD), Risikoklasse A, florider Nikotinabusus (kumulativ 20 Packungsjahre), Magenkarzinom Stadium IIA
Körperliche Untersuchung. Auskultation: beidseits vesikuläres Atemgeräusch mit gering verlängertem Exspirium und basaler Dämpfung links, keine Rasselgeräusche. Perkussion: sonorer Klopfschall bis auf links basal – hier hyposonor, Lungengrenze links nicht atemverschieblich. Herztöne rhythmisch, rein und normofrequent. Knöchelödeme"""

In [21]:
print(prompt_few_shot.format(german_text_long))

Your task is to determine the smoking status of the person described in a clinical note.
Please return the answer as a JSON of the format { label : <label>, evidence: <keyphrase> } without any explanations.
Here are some examples:
Input: ```Smoker until 1999``` Output: ```{ "label" : "ex-smoker", "keyphrase": "Smoker until 1999"}```
Input: ```… SOCIAL HISTORY: Widowed since 1972, no tobacco, no alcohol, lives alone.``` Output: ```{ "label" : "non-smoker", "keyphrase": "no tobacco"}```
Input: ```He is a heavy smoker and drinks 2–3 shots per day at times.``` Output: ```{ "label" : "current smoker", "keyphrase": "heavy smoker"}```
Input: ```
Fallschilderung
Anamnese. Der 67-jährige Patient ist bekannt in Ihrer allgemeininternistischen Praxis.
Letzte Vorstellung vor 2 Monaten: Erstdiagnose Magenkarzinom, Beginn neoadjuvante Chemotherapie (FLOT-Protokoll mit 5‑Fluorouracil, Folinsäure, Oxaliplatin und Docetaxel) in potenziell kurativem Setting im lokalen Klinikum
Heutige Vorstellung: seit e

In [22]:
get_completion(prompt_few_shot.format(german_text_long))

'{"label": "current smoker", "keyphrase": "florider Nikotinabusus"}'